**About** : This notebook is used to infer models.

In [1]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

In [2]:
cd ../src/

/workspace/kaggle_islr/src


## Initialization

### Imports

In [3]:
import os
import torch

print(torch.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"

1.14.0a0+410ce96


In [4]:
import os
import re
import cv2
import sys
import glob
import json
import time
import torch
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import *

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

In [5]:
from utils.logger import Config, upload_to_kaggle

from params import *
from data.preparation import *

from model_zoo.models import define_model
from utils.metrics import *
from utils.torch import load_model_weights
from utils.plots import plot_sample
from inference.main import uniform_soup

## Expes

In [6]:
EXP_FOLDER = "../logs/2023-04-17/42/"  # 0.7265 / x5 0.7273 / MTx10 0.7274

EXP_FOLDER = "../logs/2023-04-21/31/"
# EXP_FOLDER = "../logs/2023-04-19/6/"

EXP_FOLDER = "../logs/2023-04-23/27/"   # 0.7302 / DIST 0.7341 / DISTx10 0.7310
EXP_FOLDER = "../logs/2023-04-23/28/"   # 0.7295 / DIST 0.7341 / DISTx10 0.7348

EXP_FOLDER = "../logs/2023-04-24/9/"

In [7]:
config = Config(json.load(open(EXP_FOLDER + "config.json", "r")))

In [8]:
df = prepare_data(DATA_PATH, config.processed_folder)

In [9]:
if "fold" not in df.columns:
    folds = pd.read_csv(config.folds_file)
    df = df.merge(folds, how="left", on=["participant_id", "sequence_id"])

## Inference

### Preprocessing

In [10]:
from tflite.prepro import *

2023-04-27 14:19:32.224948: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
landmarks = np.concatenate(KEPT_LANDMARKS)
type_embed = np.zeros(1000)
start = 0
for subset, idx in zip(KEPT_LANDMARKS, MAPPING):
    print(subset, idx)
    type_embed[start: start + len(subset)] = idx
    start += len(subset)

type_embed = type_embed[type_embed > 0]

type_embed = np.concatenate([type_embed, np.array([idx] * len(TO_AVG))])

print("\nn_landmarks :", len(type_embed))

[468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488] 1
[522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542] 2
[10, 54, 67, 132, 150, 152, 162, 172, 176, 234, 284, 297, 361, 379, 389, 397, 400, 454] 3
[13, 37, 40, 61, 78, 81, 84, 87, 88, 91, 191, 267, 270, 291, 308, 311, 314, 317, 318, 321, 415] 4
[500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511] 5
[205, 425] 6

n_landmarks : 100


In [12]:
MAX_LENS = {
    "torch_12/": 25,
    "torch_16/": 30,
}

In [13]:
prepro = Preprocessing(type_embed, max_len=MAX_LENS[config.processed_folder], model_max_len=config.max_len)
config.max_len

25

In [14]:
inp = tf.keras.Input((543, 3), dtype=tf.float32, name="inputs")
prepro_tf = PreprocessingTF(type_embed, max_len=MAX_LENS[config.processed_folder], model_max_len=config.max_len)

2023-04-27 14:19:35.898491: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-04-27 14:19:35.898592: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: 4532240
2023-04-27 14:19:35.898609: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: 4532240
2023-04-27 14:19:35.898738: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 520.61.5
2023-04-27 14:19:35.898778: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 515.65.1
2023-04-27 14:19:35.898793: E tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:312] kernel version 515.65.1 does not match DSO version 520.61.5 -- cannot find working devices in this configuration


In [15]:
preds = []
times = []

# for i in tqdm(range(len(df['path']))):
for i in tqdm(range(100)):
    path = df['path'][i]
    name = f"{path.split('/')[-2]}_{path.split('/')[-1].split('.')[0]}.npy"

    pq, data = load_relevant_data_subset(path)
    
    x = torch.from_numpy(data)
    x_ = tf.constant(data)

    x_torch = prepro(x)
    x_tf = prepro_tf(x_)
    
    print(np.abs(x_tf.numpy() - x_torch.numpy()).max())

    break

  0%|          | 0/100 [00:00<?, ?it/s]

1.9073486e-06


### Model

In [16]:
from tflite.models import Model
from utils.torch import count_parameters

In [19]:
DIST = config.mt_config['distill']
DIST = True

model = Model(
    type_embed,
    embed_dim=config.embed_dim,
    transfo_dim=512 + 64, #config.transfo_dim if not DIST else 512,
    dense_dim=192,  # config.dense_dim if not DIST else 192,
    transfo_layers=3,  # config.transfo_layers  if not DIST else 2,
    transfo_heads=config.transfo_heads,
    drop_rate=config.drop_rate,
    num_classes=config.num_classes,
    max_len=config.max_len,
).cpu().eval()

config.embed_dim, config.transfo_dim

(16, 1024)

In [20]:
count_parameters(model)

6760354

In [21]:
N_SOUP = 0
TEACHER = False

In [22]:
if DIST:
    pred_oof = np.load(EXP_FOLDER + "pred_oof_dist.npy")
elif TEACHER:
    pred_oof = np.load(EXP_FOLDER + "pred_oof_teach.npy")
else:
    try:
        pred_oof = np.load(EXP_FOLDER + "pred_oof_inf.npy")
    except:
        pred_oof = np.load(EXP_FOLDER + "pred_oof.npy")

score = accuracy(df['target'], pred_oof)
print(f"-> CV acc : {score:.4f}")

-> CV acc : 0.7248


In [23]:
if N_SOUP:
    if TEACHER:
        weights = [EXP_FOLDER + f"{config.name}_teacher_fullfit_0_{ep}.pt" for ep in range(config.epochs - N_SOUP, config.epochs + 1)]
    elif DIST:
        weights = [EXP_FOLDER + f"{config.name}_distilled_fullfit_0_{ep}.pt" for ep in range(config.epochs - N_SOUP, config.epochs + 1)]
    else:
        weights = [EXP_FOLDER + f"{config.name}_fullfit_0_{ep}.pt" for ep in range(config.epochs - N_SOUP, config.epochs + 1)]
    print("-> Soup :", [w.split('/')[-1] for w in weights])
    model = uniform_soup(model, weights)

else:
    try:
    #     model = load_model_weights(model, EXP_FOLDER + f"{config.name}_fullfit_0.pt")
    #     model = load_model_weights(model, EXP_FOLDER + f"{config.name}_teacher_fullfit_0.pt")
#         model = load_model_weights(model, EXP_FOLDER + f"{config.name}_0.pt")
        model = load_model_weights(model, EXP_FOLDER + f"{config.name}_distilled_0.pt")
    
#         weights = [EXP_FOLDER + f"{config.name}_0_{ep}.pt" for ep in range(config.epochs - 10, config.epochs + 1)]
#         print("-> Soup :", [w.split('/')[-1] for w in weights])
#         model = uniform_soup(model, weights)
    
    except: # FileNotFoundError:
        print('Not loading weights !')

Not loading weights !


In [24]:
df = df[df['fold'] == 0].reset_index(drop=True)
pred_val = np.load(EXP_FOLDER + "pred_val_0.npy")
print('Ref val acc :', accuracy(df['target'], pred_val))

Ref val acc : 0.7002862510063512


In [25]:
preds = []
times = []

# for i in tqdm(range(len(df['path']))):
for i in tqdm(range(100)):
    path = df['path'][i]
    name = f"{path.split('/')[-2]}_{path.split('/')[-1].split('.')[0]}.npy"

    pq, data = load_relevant_data_subset(path)
    
    x = torch.from_numpy(data)

    t0 = time.time()
    x = prepro(x)
    y = model(x)
    preds.append(y.detach().cpu().numpy().flatten())
    t1 = time.time()
    
    times.append((t1 - t0) * 1000)
#     break

100%|██████████| 100/100 [00:37<00:00,  2.70it/s]


In [26]:
print(f'Runtime : {np.mean(times) :.1f}ms')

Runtime : 344.7ms


In [27]:
preds = np.stack(preds)

In [28]:
accuracy(df['target'].head(len(preds)), preds)

0.01

#### Nobuco

In [29]:
import nobuco
import tensorflow_addons as tfa
from nobuco import ChannelOrder, ChannelOrderingStrategy

In [30]:
@nobuco.converter(torch.nn.functional.mish, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS)
def mish(input: torch.Tensor, inplace: bool = False):
    return lambda input, inplace=False: tfa.activations.mish(input)

In [31]:
@nobuco.converter(torch.Tensor.long, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS)
def long(input: torch.Tensor, inplace: bool = False):
    return lambda input, inplace=False: tf.cast(input, tf.int64)

In [32]:
@nobuco.converter(torch.Tensor.int, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS)
def int(input: torch.Tensor, inplace: bool = False):
    return lambda input, inplace=False: tf.cast(input, tf.int32)

In [33]:
@nobuco.converter(torch.Tensor.amax, channel_ordering_strategy=ChannelOrderingStrategy.FORCE_PYTORCH_ORDER)
def amax(input: torch.Tensor, dim=None, keepdim=False):
    return lambda input, axis: tf.reduce_max(input, axis=dim)

In [34]:
@nobuco.converter(torch.gather, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS)
def gather(input, dim, index):
    return lambda input, dim, index: tf.gather(input, index)

In [35]:
@nobuco.converter(torch.zeros, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS)
def zeros(*size):
    print(size)
    return lambda size: tf.zeros(size)

In [36]:
path = df['path'][0]
pq, data = load_relevant_data_subset(path)
inp = torch.from_numpy(prepro_tf(data).numpy()).contiguous()
inp.size()

torch.Size([14, 5, 100])

In [37]:
keras_model = nobuco.pytorch_to_keras(
    model,
    args=[inp],
    input_shapes={inp: (None, 5, 100)},
    inputs_channel_order=ChannelOrder.PYTORCH,
    outputs_channel_order=ChannelOrder.TENSORFLOW
)

Legend:
    Green — conversion successful
    Yellow — conversion imprecise
    Red — conversion failed
    Red — no converter found
    Bold — conversion applied directly
    * — subgraph reused
    Tensor — this output is not dependent on any of subgraph's input tensors
    Tensor — this input is a parameter / constant
    Tensor — this tensor is useless

Model[tflite.models](float32_0<14,5,100>) -> float32_673<1,250>
 │  unsqueeze[torch.Tensor](float32_0<14,5,100>, 0) -> float32_1<1,14,5,100>
 │  shape[nobuco.funcs](float32_1<1,14,5,100>) -> (int32_2<>, int32_3<>, int32_4<>, int32_5<>)
 │  __getitem__[torch.Tensor](float32_1<1,14,5,100>, (:, :, 0)) -> float32_6<1,14,100>
 │  long[torch.Tensor](float32_6<1,14,100>) -> int64_7<1,14,100>
 │  Embedding[torch.nn.modules.sparse](int64_7<1,14,100>) -> float32_9<1,14,100,16>
 │   ├· embedding[torch.nn.functional](int64_7<1,14,100>, float32_8<9,16>, 0, None, 2.0, False, False) -> float32_9<1,14,100,16>
 │   └   └· embedding[torch](float32_8<

In [38]:
# keras_model.summary()

In [40]:
preds = []
for i in tqdm(range(100)):
    path = df['path'][i]
    pq, data = load_relevant_data_subset(path)
    x = prepro_tf(data)
    y = keras_model(x)
    preds.append(y.numpy()[0])
#     break

100%|██████████| 100/100 [00:39<00:00,  2.56it/s]


In [41]:
accuracy(df['target'].head(len(preds)), preds)

0.01

### Prepro + model

In [68]:
class TFLiteModel(tf.keras.Model):
    def __init__(self, prepro, model):
        super(TFLiteModel, self).__init__()
        self.prepro = prepro
        self.model = model
    
    @tf.function(input_signature=[tf.TensorSpec(shape=[None, 543, 3], dtype=tf.float32, name='inputs')])
    def call(self, inputs=None):
        x = self.prepro(tf.cast(inputs, dtype=tf.float32))
        y = self.model(x)

        return {'outputs': y}

In [69]:
prepro_tf = PreprocessingTF(type_embed, max_len=MAX_LENS[config.processed_folder], model_max_len=config.max_len)

In [70]:
tflite_keras_model = TFLiteModel(prepro_tf, keras_model)

In [71]:
preds = []

# for i in tqdm(range(len(df['path']))):
for i in tqdm(range(1000)):
    path = df['path'][i]
    name = f"{path.split('/')[-2]}_{path.split('/')[-1].split('.')[0]}.npy"

    pq, data = load_relevant_data_subset(path)

    y = tflite_keras_model(data)
    
    preds.append(y['outputs'].numpy()[0])
#     break

100%|██████████| 1000/1000 [00:20<00:00, 49.90it/s]


In [72]:
accuracy(df['target'].head(len(preds)), preds)

AttributeError: 'bool' object has no attribute 'mean'

In [73]:
tflite_keras_model.save(EXP_FOLDER + 'model_keras')

2023-04-27 14:38:35.000913: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,543,3]
	 [[{{node inputs}}]]
2023-04-27 14:38:35.001109: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'tf_lite_model_4/244713' with dtype int32 and shape [1000]
	 [[{{node tf_lite_model_4/244713}}]]
2023-04-27 14:38:35.526329: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_1' with dtype float and shape [?,543,3]
	 

2023-04-27 14:38:42.194552: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,543,3]
	 [[{{node inputs}}]]
2023-04-27 14:38:42.194734: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor '251027' with dtype int32 and shape [1000]
	 [[{{node 251027}}]]


2023-04-27 14:38:42.690650: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,543,3]
	 [[{{node inputs}}]]
2023-04-27 14:38:42.690865: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor '251058' with dtype int32 and shape [1000]
	 [[{{node 251058}}]]


2023-04-27 14:38:43.465375: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'x' with dtype float and shape [?,543,3]
	 [[{{node x}}]]
2023-04-27 14:38:43.465599: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor '251087' with dtype int32 and shape [1000]
	 [[{{node 251087}}]]
2023-04-27 14:38:45.404833: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,5,100]
	 [[{{node inputs}}]]
2023-04-27 14:38:45.642

INFO:tensorflow:Assets written to: ../logs/2023-04-24/9/model_keras/assets


INFO:tensorflow:Assets written to: ../logs/2023-04-24/9/model_keras/assets


### TfLite

In [74]:
converter = tf.lite.TFLiteConverter.from_saved_model(EXP_FOLDER + "model_keras")

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

# converter.target_spec.supported_ops = [
#     tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
#     tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
# ]

tflite_model = converter.convert()

with open(EXP_FOLDER + 'model.tflite', 'wb') as f:
    f.write(tflite_model)

2023-04-27 14:39:09.782232: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'serving_default_inputs' with dtype float and shape [?,543,3]
	 [[{{node serving_default_inputs}}]]
2023-04-27 14:39:10.869505: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-04-27 14:39:10.869578: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-04-27 14:39:10.870895: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: ../logs/2023-04-24/9/model_keras
2023-04-27 14:39:10.915262: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-04-27 14:39:10.915325: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: ../logs/2023-04-24/9/mo

In [75]:
from tflite_runtime.interpreter import Interpreter

interpreter = Interpreter(EXP_FOLDER + "model.tflite")

prediction_fn = interpreter.get_signature_runner("serving_default")

output = prediction_fn(inputs=data)
output['outputs'].max()

100.0

In [76]:
preds = []
times = []
# for i in tqdm(range(len(df['path']))):
for i in tqdm(range(1000)):
    path = df['path'][i]
    name = f"{path.split('/')[-2]}_{path.split('/')[-1].split('.')[0]}.npy"

    pq, data = load_relevant_data_subset(path)

    t0 = time.time()
    output = prediction_fn(inputs=data)
    t1 = time.time()

    preds.append(output['outputs'])
    times.append((t1 - t0) * 1000)
    
#     break

100%|██████████| 1000/1000 [00:16<00:00, 59.04it/s]


In [ ]:
accuracy(df['target'].head(len(preds)), preds)

In [ ]:
accuracy(df['target'].head(len(preds)), pred_val[:len(preds)])

In [77]:
print(f'-> Runtime : {np.mean(times) :.1f}ms')

if np.mean(times) > 100:
    print("\n WARNING ! Runtime must be < 100 ms !")

-> Runtime : 1.3ms


In [53]:
print(f'-> Runtime : {np.mean(times) :.1f}ms')

if np.mean(times) > 100:
    print("\n WARNING ! Runtime must be < 100 ms !")

-> Runtime : 30.9ms


### Size & upload

In [54]:
size = os.path.getsize(EXP_FOLDER + 'model.tflite') / np.power(1024, 2)
print(f"-> Model size : {size:.3f} Mo")

assert size < 40, "Model size must be < 40 Mo !"

-> Model size : 13.448 Mo


In [55]:
upload_to_kaggle([EXP_FOLDER], "/workspace/datasets/islr_weights_1/", "ISLR Models", update_folders=False)

- Copying ../logs/2023-04-24/9/ ...

Dataset size : 0.236 Go
- Update existing dataset !
- Uploading ...


100%|██████████| 39.0M/39.0M [00:10<00:00, 4.02MB/s]
100%|██████████| 27.2M/27.2M [00:06<00:00, 4.66MB/s]
100%|██████████| 22.0/22.0 [00:01<00:00, 12.6B/s]
100%|██████████| 28.5M/28.5M [00:06<00:00, 4.40MB/s]
100%|██████████| 20.4M/20.4M [00:04<00:00, 5.29MB/s]
100%|██████████| 13.4M/13.4M [00:02<00:00, 5.31MB/s]
100%|██████████| 35.8M/35.8M [00:09<00:00, 4.15MB/s]
100%|██████████| 28.5M/28.5M [00:06<00:00, 4.63MB/s]
100%|██████████| 10.0M/10.0M [00:02<00:00, 3.80MB/s]
100%|██████████| 39.0M/39.0M [00:13<00:00, 2.92MB/s]  


Output :
 b'Starting upload for file ens_nobuco2_model.tflite\nUpload successful: ens_nobuco2_model.tflite (39MB)\nStarting upload for file 2023-04-17_42_model.tflite\nUpload successful: 2023-04-17_42_model.tflite (27MB)\nStarting upload for file .ipynb_checkpoints.zip\nUpload successful: .ipynb_checkpoints.zip (22B)\nStarting upload for file 2023-04-12_18_model.tflite\nUpload successful: 2023-04-12_18_model.tflite (28MB)\nStarting upload for file 2023-04-18_10_model.tflite\nUpload successful: 2023-04-18_10_model.tflite (20MB)\nStarting upload for file 2023-04-24_9_model.tflite\nUpload successful: 2023-04-24_9_model.tflite (13MB)\nStarting upload for file ens_distilled_model.tflite\nUpload successful: ens_distilled_model.tflite (36MB)\nStarting upload for file 2023-04-13_29_model.tflite\nUpload successful: 2023-04-13_29_model.tflite (28MB)\nStarting upload for file 2023-04-12_2_model.tflite\nUpload successful: 2023-04-12_2_model.tflite (10MB)\nStarting upload for file ens_nobuco1_mode

Done ! 